# 实验说明

本实验基于`naturalcc`工具集，它是业界首个专注代码智能的深度模型开源训练工具集，包含了对业界现有的代码模型和下游任务的性能评价。相关论文发表在软件工程顶级会议ICSE上，参考文献 [NaturalCC: An Open-Source Toolkit for Code Intelligence](https://xcodemind.github.io/papers/icse22_naturalcc_camera_submitted.pdf)，实验要求使用`naturalcc`工具集复现类型推导任务中的经典方法`Typilus`，此方法发表在2020年的程序语言顶级会议PLDI上，相关文献为[Typilus: neural type hints](https://arxiv.org/pdf/2004.10657.pdf)。

实验分为五个部分，环境准备、数据获取、模型和超参数配置、模型训练与评价、拓展研究。

完整复现Typilus需要计算节点的内存大于或等于`128GB`，显存大于或等于 `32GB`，如无计算节点，建议裁剪数据集，以及使用 [Google Colab](http://colab.research.google.com) (可以提供12GB的内存和一张16GB显存的 Tesla T4显卡）

(colab不支持docker，所以不建议使用）

实验只能在Linux系统上运行，可以选择安装双系统、VMWare虚拟机或WSL子系统（Windows Subsystem for Linux）进行实验。

为避免包冲突，建议使用[Anaconda](http://anaconda.org)管理Python环境。

## NaturalCC 环境

首先在GitHub上下载`naturalcc`工具集，执行下面的命令

In [6]:
!git clone https://github.com/CGCL-codes/naturalcc.git

Cloning into 'naturalcc'...
remote: Enumerating objects: 2740, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 2740 (delta 36), reused 33 (delta 9), pack-reused 2638
Receiving objects: 100% (2740/2740), 82.38 MiB | 7.65 MiB/s, done.
Resolving deltas: 100% (1135/1135), done.


之后在本地安装naturalcc环境，需要提前安装pytorch，此命令只需要执行一次。
推荐使用 Anaconda 虚拟环境容器，避免影响本地Python环境。Anaconda可以在[这里](https://www.anaconda.com/)获取。

Python要求最低版本3.8，安装完成后，执行下面的命令

In [1]:
%cd naturalcc
!pip install -r requirements.txt
!pip install --editable .
%cd ..

/home/dell/Code/jupyter/naturalcc


### [GPU训练] 需要注意的地方

训练过程依赖[pytorch库](https://pytorch.org)和[dgl库](https://www.dgl.ai)，如果使用GPU+CUDA训练，需要确保所安装的库版本与本机的CUDA版本对应，例如CUDA 11.0+需要安装```torch+cu110, dgl-cu110```，如果仅使用CPU训练，则只需要安装CPU版本的`torch`和`dgl`即可

注意：conda安装pytorch可能只会安装cpu版本（无论是否选择gpu包），因此建议使用pip安装。如果你有gpu，则推荐的安装方法：

~~~shell
pip3 install torch==1.12+cu113 --no-cache-dir --extra-index-url https://download.pytorch.org/whl/cu113
conda install -c dglteam dgl-cuda11.3
~~~

其中`torch==1.12`为pytorch的版本（我用的是1.12，注意不要用最新版1.13），`cu113`和`dgl-cuda11.3`对应于你自己本机的CUDA版本（可以使用命令`nvcc -V`查看，我用的是CUDA==11.3，要注意pytorch较新的版本不再支持CUDA 10及以下），注意`--extra-index-url`里面也有一个cu113要改

在使用GPU训练前需要通过`gpustat`或`nvidia-smi`检查GPU是否已经正确安装，如下所示

In [4]:
!gpustat

dell-gpu             Wed Oct 12 11:02:22 2022  460.56
[0] GeForce RTX 3090 | 29'C,   0 % |     0 / 24268 MB |
[1] GeForce RTX 3090 | 31'C,   0 % |     0 / 24268 MB |


## Typilus环境

构建Typilus数据集需要使用`docker`，在电脑上安装`docker`后，获取`typilus`的源代码

In [1]:
!git clone https://github.com/typilus/typilus.git

Cloning into 'typilus'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 159 (delta 38), reused 111 (delta 15), pack-reused 0
Receiving objects: 100% (159/159), 162.86 KiB | 2.06 MiB/s, done.
Resolving deltas: 100% (38/38), done.


之后在 `typilus/src/data_preparation`目录构建docker镜像

In [ ]:
%cd typilus/src/data_preparation/
!docker build -t typilus-env .

> 注：在linux系统运行docker需要root用户权限。如果是在实验室的服务器上运行，则需要事先向管理员索要root权限，或通过管理员加入docker用户组。

# 数据获取

## Typilus Graph 构建

在开始训练之前，我们首先需要获得训练所需的数据，我们将首先使用`Typilus`内置的数据处理程序将Python数据转换为Typilus Graph，然后把graph导入`naturalcc`进行训练和预测

运行刚才构建的docker镜像，并设置Typilus训练数据保存的位置

In [ ]:
!docker run --rm -it -v xxx/yyy/zzz:/usr/data typilus-env:latest bash

> `docker run` 是启动 docker 容器的命令。参数`-it`表示使用交互模式，新建一个终端。`-v xxx/yyy/zzz:/usr/data`表示挂载主机目录到容器的目录，当你指定一个主机目录为`xxx/yyy/zzz`的时候，就可以在主机上的这个文件夹里面看到docker容器中`/usr/data`路径下的文件了。所以这里的`xxx/yyy/zzz`要替换成你自己的机器上一个空间比较大的位置。
注意：一旦关闭容器所有的数据就会丢失，这些流程就要重新来过，所以不要关闭容器！

参数`--rm`表示stop container后会自动删掉这个container, 在windows环境中需要给
-v xxx/yyy/zzz:/usr/data添加双引号"", xxx/yyy/zzz为windows上想要指定得主机目录。typilus-env:latest的:后的是想创建的image的标签

在Docker shell中，输入下面的命令来构建数据集

``bash scripts/prepare_data.sh metadata/typedRepos.txt``

*注意：这条命令可能会执行若干天，因为需要从github clone500余个代码存储库（这需要大量的存储空间）。关于命令执行时间过长和死循环的问题，可以参考[这里](https://github.com/typilus/typilus/issues/1)*

以下是我的暴力解决方案，仅供参考：

如果卡顿很久不再继续，可以手动 Ctrl+c 杀死当前的进程。如果后面又会卡掉，多这样做几次再停掉。会有一部分数据集构建成功。

实际上，你可以在`typilus/src/data_preparation/scripts/`下发现卡住的脚本`prepare_data.sh`，它会卡在第35行开始的循环内：

~~~shell
# Create dataset spec
for repo in ./*; do
    cd ./$repo
    echo $(git remote get-url origin) $(git rev-parse HEAD) >> ../../dataset.spec
    cd ..
done
~~~

如果你手动停掉了这个循环，那么你需要把脚本`prepare_data.sh`在该循环后面的命令手动执行一遍。注意，第41行`cd ..`之后所在的目录是`/usr/data`（不是你自己机子的目录，是docker容器内的目录）。

这两个循环之后的过程是不会卡住的（用pytype进行类型分析的过程需要较长时间，但最终会执行完毕，只要你肯等）

获取的数据保存在 `xxx/yyy/zzz/graph-dataset-split` （本机地址）`/usr/data/graph-dataset-split`（docker容器地址）中，使用Tree命令观察

（若在docker容器中执行这一步，则要先在docker容器中安装tree：`sudo apt-get install tree`）

In [8]:
!tree /mnt/gold/bizq/Typilus_data/graph-dataset-split

/mnt/gold/bizq/Typilus_data/graph-dataset-split
├── test
│   ├── graph-000.jsonl.gz
│   ├── graph-001.jsonl.gz
│   ├── graph-002.jsonl.gz
│   ├── graph-003.jsonl.gz
│   ├── graph-004.jsonl.gz
│   ├── graph-005.jsonl.gz
│   ├── graph-006.jsonl.gz
│   ├── graph-007.jsonl.gz
│   ├── graph-008.jsonl.gz
│   ├── graph-009.jsonl.gz
│   ├── graph-010.jsonl.gz
│   ├── graph-011.jsonl.gz
│   ├── graph-012.jsonl.gz
│   ├── graph-013.jsonl.gz
│   ├── graph-014.jsonl.gz
│   ├── graph-015.jsonl.gz
│   ├── graph-016.jsonl.gz
│   ├── graph-017.jsonl.gz
│   ├── graph-018.jsonl.gz
│   ├── graph-019.jsonl.gz
│   ├── graph-020.jsonl.gz
│   ├── graph-021.jsonl.gz
│   ├── graph-022.jsonl.gz
│   ├── graph-023.jsonl.gz
│   ├── graph-024.jsonl.gz
│   ├── graph-025.jsonl.gz
│   ├── graph-026.jsonl.gz
│   ├── graph-027.jsonl.gz
│   ├── graph-028.jsonl.gz
│   ├── graph-029.jsonl.gz
│   └── graph-030.jsonl.gz
├── train
│   ├── graph-000.jsonl.gz
│   ├── graph-001.jsonl.gz
│   ├── graph-002.jsonl.gz
│   ├── graph-0

## NaturalCC 数据处理

接下来把生成的Typilus Graph导入NaturalCC

如果上述过程执行成功，这一步开始就不用docker了，可以另开一个终端执行（先不要急着关闭docker容器！关闭容器后如果再想对数据集操作就需要从头开始！）

数据的处理过程包含了两个阶段，首先将原始数据整理为naturalcc统一的格式，接下来对数据进行binarize，以适合模型训练。

>首先，找到typilus配置文件`naturalcc/ncc_dataset/typilus/preprocess/typilus.yml`，修改以下几项为：
> ~~~yaml
>  trainpref: ~/typilus/attributes/train #", metavar="FP", default=None, help="train file prefix"
>  validpref: ~/typilus/attributes/valid #", metavar="FP", default=None, help="comma separated, valid file prefixes"
>  testpref:  ~/typilus/attributes/test  #", metavar="FP", default=None, help="comma separated, test file prefixes"
>~~~
>以及（去掉并行运行）
>~~~yaml
>  workers: 1 # ", metavar="N", default=1, type=int, help="number of parallel workers"
>~~~

以上两步已经在新的代码中被修复了，不用管。

然后运行（会有点慢）：

> 以下三行代码可以放入一个.py文件，然后使用`python xxx.py`执行。函数`prepare_dataset()`中的参数`typilus_path`改成你自己构建数据集所在的目录，也就是`./graph-dataset-split/`的上级目录
> 注意：这个文件应放在之前git clone得到的`naturalcc`文件夹下，并以该文件夹为工作目录执行！

In [3]:
import ncc_dataset
ncc_dataset.prepare_dataset('typilus', typilus_path="./data")

[2022-10-16 08:59:19]    INFO >> The typilus graph is migrated to NaturalCC directory. 
To re-migrate, please delete the directory '/mnt/gold/bizq/ncc_data/typilus/raw' (dataset_migration.py:24, dataset_migration())
[2022-10-16 08:59:19]    INFO >> The typilus dataset is already flattened, to re-flatten the dataset, please delete the directory '/mnt/gold/bizq/ncc_data/typilus/attributes'. (flatten.py:43, flatten())


In [5]:
ncc_dataset.binarize_dataset('typilus')

NaturalCC dataset and cache path: '/mnt/gold/bizq/ncc_data'
Using backend: pytorch
[2022-10-16 09:30:49]    INFO >> Namespace(yaml_file='typilus') (preprocess.py:418, cli_main())
[2022-10-16 09:30:49]    INFO >> Load arguments in /home/dell/Code/jupyter/naturalcc/ncc_dataset/typilus/preprocess/typilus.yml (preprocess.py:420, cli_main())
[2022-10-16 09:30:49]    INFO >> {'preprocess': {'task': 'typilus', 'langs': ['nodes', 'edges', 'supernodes.annotation'], 'trainpref': '/mnt/gold/bizq/ncc_data/typilus/attributes/train', 'validpref': '/mnt/gold/bizq/ncc_data/typilus/attributes/valid', 'testpref': '/mnt/gold/bizq/ncc_data/typilus/attributes/test', 'dataset_impl': 'mmap', 'destdir': '/mnt/gold/bizq/ncc_data/typilus/type_inference/data-mmap', 'only_train': 1, 'edge_backward': 1, 'thresholds': [5, 5, 5], 'dicts': [None, None, None], 'nwords': [9999, 99, 99], 'padding_factor': 1, 'workers': 40}} (preprocess.py:422, cli_main())
[2022-10-16 09:30:49]    INFO >> mkdir /mnt/gold/bizq/ncc_data/ty

以下是我踩过的坑：
* FileNotFoundError: The dataset variable $NCC is not set, please first define the variable.
解决方案：在linux终端运行命令`export NCC=xxx/yyy/zzz/data`，即设置环境变量NCC为你刚构建好的数据集的目录
* ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version 'GLIBCXX_3.4.26' not found (required by <...>/envs/typilus/lib/python3.9/site-packages/scipy/linalg/_matfuncs_sqrtm_triu.cpython-39-x86_64-linux-gnu.so)
解决方案：参考[这里](https://blog.csdn.net/weixin_36488777/article/details/116897183)，在环境变量LD_LIBRARY_PATH中用新的GCC库覆盖原来的GCC库。
* RuntimeError: An attempt has been made to start a new process before the current process has finished its bootstrapping phase. This probably means that you are not using fork to start your child processes and you have forgotten to use the proper idiom in the main module: ...
解决方案：多进程设置不能再使用了。把typilus配置文件`naturalcc/ncc_dataset/typilus/preprocess/typilus.yml`中最后一行的workers的40改成1（也就是上面的那一步）。


# 模型训练

我们已经提供了训练代码，模型训练的过程只需要执行代码即可。这一步需要大量显存，建议裁剪数据集之后再进行训练。

在训练前，需要在当前conda环境安装scikit-learn：`pip install scikit-learn`

如果需要指定训练的GPU（默认0号卡），在`naturalcc/run/type_prediction/typilus/config/typilus.yml`中找到键`device_id`，并修改它。

In [2]:
!python naturalcc/run/type_prediction/typilus/train.py

Using backend: pytorch
[2022-10-17 15:35:01]    INFO >> Load arguments in naturalcc/run/type_prediction/typilus/config/typilus.yml (train.py:295, cli_main())
[2022-10-17 15:35:01]    INFO >> {'criterion': 'typilus', 'optimizer': 'torch_adam', 'lr_scheduler': 'fixed', 'tokenizer': None, 'bpe': None, 'common': {'no_progress_bar': 0, 'log_interval': 50, 'log_format': 'simple', 'tensorboard_logdir': '', 'memory_efficient_fp16': 1, 'fp16_no_flatten_grads': 1, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'empty_cache_freq': 0, 'task': 'typilus', 'seed': 1, 'cpu': 0, 'fp16': 0, 'fp16_opt_level': '01', 'server_ip': '', 'server_port': '', 'bf16': 0}, 'dataset': {'num_workers': 0, 'skip_invalid_size_inputs_valid_test': 1, 'max_tokens': None, 'max_sentences': 32, 'required_batch_size_multiple': 8, 'dataset_impl': 'mmap', 'train_subset': 'train', 'valid_subset': 'valid', 'validate_interval': 1, 'fixed_valida

In [15]:
!gpustat

dell-gpu             Sun Oct 16 14:47:50 2022  460.56
[0] GeForce RTX 3090 | 34'C,   0 % |     0 / 24268 MB |
[1] GeForce RTX 3090 | 39'C,   0 % |     0 / 24268 MB |


# 模型评价

**挑战1**. 我们在训练代码中插入了模型的评价代码，试着用它们评价模型的准确率！

# 拓展研究

**挑战2**. 调整Typilus模型的超参数（在`config/typilus.yml`中），试着提高模型的训练准确率。

**挑战3**. 利用`naturalcc`中的其他代码，比较在同样的数据集上，`lstm`,`transformer`,`typilus` 的模型预测能力。

**挑战4**. 修改naturalcc的代码，支持[LAMBDANET](https://arxiv.org/pdf/2005.02161.pdf), [Type4Py](https://arxiv.org/pdf/2101.04470.pdf), [Plato](https://arxiv.org/pdf/2107.00157.pdf) 等模型。